In [1]:
### COMTRADE DATASETS EXTRACTING

In [2]:
### RUN EVERY TIME: INITIALIZATION

import pandas as pd
import numpy as np
import requests
import gc
import os
import time

In [3]:
### DISABLING OF WARNINGS

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [18]:
### CONSTANTS (RESEARCH VERSION ONLY)

### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### NA for MS Excel files:
list_na_excel_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null',
                        '#N/A Requesting Data...', '#N/A Invalid Security', '#N/A Field Not Applicable', '---']
### UN Comtrade adopted data containers:
str_path_unc_res_all_annual = 'Data_Files/Source_Files/unc_res_all_annual.h5'
str_path_unc_res_services_annual = 'Data_Files/Source_Files/unc_res_services_annual.h5'
str_key_unc_res = 'unc_res'
### File with aggregated flows:
str_path_unc_res_flows = 'Data_Files/Source_Files/unc_res_flows.h5'
### Universal HDF5 key:
str_key_unc_res = 'unc_res'
### Augmented bilateral export:
str_path_export_bilateral = 'Data_Files/Source_Files/comtrade_export_bilateral.h5'
### Export key:
str_key_unc_export = 'export_augmented'
### Augmented bilateral import:
str_path_import_bilateral = 'Data_Files/Source_Files/comtrade_import_bilateral.h5'
### Import key:
str_key_unc_import = 'import_augmented'

In [5]:
### MAIN CONSTANTS

### Dates:
date_start = pd.Timestamp('1989-12-29')
date_end = pd.Timestamp('2022-12-30')

In [6]:
### DEFINING & LAUNCH COUNTRY CODES EXTRACTOR (RESEARCH VERSION ONLY)

def get_country_codes(use_local_copy = False):  
    ### In case if URL is unavailable:
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    ### Online extraction:
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']].sort_index()    
    df_result.index = df_result.index.str.upper()
    ### Results output:
    return df_result

### World Country Codes:
df_country_codes = get_country_codes()

In [7]:
### UN COMTRADE: CODELISTS LOADING (PRODUCT VERSION)

### Codelists container:
dict_codelist = {}
### List of reference tables for categories (request parameters) loading:
request_session = requests.Session()
obj_unc_reference = request_session.get('https://comtradeapi.un.org/files/v1/app/reference/ListofReferences.json')
df_cat_reference = pd.DataFrame(obj_unc_reference.json()['results']).set_index('category')
### Parameters:
ser_type_code = pd.Series(['Goods', 'Services'], index = ['C', 'S'])
ser_type_code.index.names = ['id']
ser_type_code.name = 'typeCode'
dict_codelist['typeCode'] = ser_type_code
### Commodity HS Codes:
df_hs_comm = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:HS']['fileuri']).json()['results'])
### Filtering needed level of coomodity groups aggregation:
ser_hs_comm_ag2 = df_hs_comm[df_hs_comm['aggrLevel'] == 2].drop(['parent', 'isLeaf', 'aggrLevel'], axis = 1).set_index('id').squeeze().str[5: ]
ser_hs_comm_ag2.name = 'clCode'
dict_codelist['clCode'] = {}
dict_codelist['clCode']['C'] = ser_hs_comm_ag2
### Service EBOPS Codes:
df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB']['fileuri']).json()['results'])
### Filtering needed level of services groups aggregation:
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
ser_eb_serv_ag2 = df_eb_serv_ag2[df_eb_serv_ag2['id'].astype(int) <= 950].set_index('id')['text']
ser_eb_serv_ag2.name = 'clCode'
dict_codelist['clCode']['S'] = ser_eb_serv_ag2
### United codes:
dict_codelist['clCode']['T'] = pd.concat([dict_codelist['clCode']['C'], dict_codelist['clCode']['S']], axis = 0)
### Reporter Codes:
df_reporter_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['reporter']['fileuri']).json()['results'])
df_reporter_raw['id'] = df_reporter_raw['id'].astype(str).str.zfill(3)
df_reporter_raw['entryEffectiveDate'] = pd.to_datetime(df_reporter_raw['entryEffectiveDate'])
df_reporter_raw['entryExpiredDate'] = pd.to_datetime(df_reporter_raw['entryExpiredDate'])
### Reporters filtering to exclude aggregated and regional values (need to be replaced with SQL-based code):
df_reporter_raw = df_reporter_raw[df_reporter_raw['reporterCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
### Non-actual country codes filtering out:
df_reporter_raw = df_reporter_raw[df_reporter_raw['entryExpiredDate'].isna() | (df_reporter_raw['entryExpiredDate'] > date_start)]
ser_reporter_code = df_reporter_raw.set_index('id')['reporterCodeIsoAlpha2'].squeeze()
ser_reporter_code.name = 'Reporter'
dict_codelist['reporterCode'] = ser_reporter_code
### Partner Codes:
df_partner_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['partner']['fileuri']).json()['results'])
df_partner_raw['id'] = df_partner_raw['id'].astype(str).str.zfill(3)
df_partner_raw['entryEffectiveDate'] = pd.to_datetime(df_partner_raw['entryEffectiveDate'])
df_partner_raw['entryExpiredDate'] = pd.to_datetime(df_partner_raw['entryExpiredDate'])
### Partners filtering to exclude aggregated and regional values (need to be replaced with SQL-based code):
df_partner_raw = df_partner_raw[df_partner_raw['PartnerCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
### Non-actual country codes filtering out:
df_partner_raw = df_partner_raw[df_partner_raw['entryExpiredDate'].isna() | (df_partner_raw['entryExpiredDate'] > date_start)]
ser_partner_code = df_partner_raw.set_index('id')['PartnerCodeIsoAlpha2'].squeeze()
ser_partner_code.name = 'Partner'
dict_codelist['partnerCode'] = ser_partner_code
### Trade Flow Codes:
ser_flow_code = pd.DataFrame(request_session.get(df_cat_reference.loc['flow']['fileuri']).json()['results']).set_index('id').squeeze()
ser_flow_code.name = 'flowCode'
dict_codelist['flowCode'] = ser_flow_code

In [9]:
### UN COMTRADE: DATA REQUEST EXECUTION

def get_un_comtrade_data(str_type, str_freq, str_classification, str_trade_flow, list_reporters, list_partners, list_commodities, list_periods, str_api_key, 
                         int_limit = 99999):
    ### Request preparation:
    str_url_base = 'https://comtradeapi.un.org/data/v1/get/'
    str_url_request = str_url_base + str_type + '/'
    str_url_request = str_url_request + str_freq + '/' 
    str_url_request = str_url_request + str_classification + '?'   
    str_url_request = str_url_request + 'flowCode=' + str_trade_flow 
    str_url_request = str_url_request + '&reporterCode=' + ','.join(list_reporters)
    str_url_request = str_url_request + '&partnerCode=' + ','.join(list_partners)    
    str_url_request = str_url_request + '&cmdCode=' + ','.join(list_commodities)
    str_url_request = str_url_request + '&customsCode=' + 'C00'
    str_url_request = str_url_request + '&motCode=' + '0'    
    str_url_request = str_url_request + '&partner2Code=' + '000'       
    str_url_request = str_url_request + '&period=' + ','.join(list_periods)  
    str_url_request = str_url_request + '&maxrecords=' + str(int_limit)
    ### Request sending:
    bool_loaded = False
    while (not bool_loaded):
        request_session = requests.Session()
        dict_request_headers = {}
        dict_request_headers['Cache-Control'] = 'no-cache'
        dict_request_headers['Ocp-Apim-Subscription-Key'] = str_api_key
        request_session.headers.update(dict_request_headers)
        ### Respond processing:
        print(str_url_request)    
        obj_unc_dataset = request_session.get(str_url_request)
        print(obj_unc_dataset.json())
        ### Request error marker:
        if not ('count' in obj_unc_dataset.json()):
#            print(obj_unc_dataset.json())
            print(obj_unc_dataset.json()['error'])
            int_dataset_length = -1
        else:
            int_dataset_length = obj_unc_dataset.json()['count']
        ### Respond result transformation:
        if (int_dataset_length > 0):
            df_dataset_raw = pd.DataFrame(obj_unc_dataset.json()['data'])
            ### Selecting columns:
            df_dataset_res = df_dataset_raw[['flowCode', 'typeCode', 'period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]        
            ### Replacing Flow codes to Flow names with categorization:
            df_dataset_res.loc[:, 'Flow'] = df_dataset_res['flowCode'].replace(dict_codelist['flowCode']).astype('category').values
            ### Expanding categorical list to full list of possible values:
            df_dataset_res['Flow'].cat.set_categories(sorted(dict_codelist['flowCode'].values), ordered = True, inplace = True)
            ### Replacing flow codes to flow names with categorization:
            df_dataset_res.loc[:, 'Type'] = df_dataset_res['typeCode'].replace(dict_codelist['typeCode']).astype('category').values
            ### Expanding categorical list to full list of possible values:
            df_dataset_res['Type'].cat.set_categories(sorted(dict_codelist['typeCode'].values), ordered = True, inplace = True)
            ### Year to Date transformation:
            df_dataset_res.loc[:, 'Date'] = (pd.to_datetime(df_dataset_raw['period']) + pd.offsets.BYearEnd()).values
            ### Replacing Reporter codes to ISON IDs with categorization:
            df_dataset_res.loc[:, 'Reporter'] = df_dataset_res['reporterCode'].astype(str).str.zfill(3).replace(dict_codelist['reporterCode'])\
                                                                                                       .astype('category').values
            ### Expanding categorical list to full list of possible values:
            df_dataset_res['Reporter'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            ### Replacing Partner codes to ISON IDs with categorization:
            df_dataset_res.loc[:, 'Partner'] = df_dataset_res['partnerCode'].astype(str).str.zfill(3).replace(dict_codelist['partnerCode']).astype('category').values
            ### Expanding categorical list to full list of possible values:
            df_dataset_res['Partner'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            ### Commodity Type categorization:
            df_dataset_res.loc[:, 'Commodity_ID'] = df_dataset_res['cmdCode'].astype('category').values
            ### Expanding categorical list to full list of possible values:
            df_dataset_res['Commodity_ID'].cat.set_categories(sorted(dict_codelist['clCode']['T'].index), ordered = True, inplace = True)
            ### Values scaling and transformation to integer:
            df_dataset_res.loc[:, 'Value'] = (df_dataset_res['primaryValue'] / 1000).astype('int32').values
            ### Data clearing:
            df_dataset_res.drop(df_dataset_res[(df_dataset_res['Reporter'] == 'SA') & (df_dataset_res['Partner'] == 'TW')].index, inplace = True)
            df_dataset_res.drop(df_dataset_res[df_dataset_res['Reporter'] == df_dataset_res['Partner']].index, inplace = True)
            df_dataset_res['Value'].clip(lower = 0, inplace = True)
            ### Dropping extra columns:
            df_dataset_res = df_dataset_res[['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID', 'Value']]#.dropna()
            print('Loaded Observations Number:', int_dataset_length)
            bool_loaded = True
        elif (int_dataset_length == 0):
            print('Empty Dataset')
            bool_loaded = True            
            df_dataset_res = None            
        else:
            print('Loading Error. Let\'s try once more...')
    return df_dataset_res

In [8]:
### UN COMTRADE: GOODS DATA REQUEST PARAMETERS

### Primary key to authorize:
#str_primary_key = 'e690550ab9414234a6b705220596677a'
#str_primary_key = 'd79c218e2e9d464fade1810fd14347d8'
str_primary_key = '3f0b8d53d71b401e840d58c90a283176'
### Type: Goods
str_goods_type = 'C'
### Type: Services
str_services_type = 'S'
### Annual frequency:
str_freq = 'A'
### Goods classification:
str_goods_class = 'HS'
### Services classification classification:
str_services_class = 'EB'
### Flow: Export
str_export_flow = 'X'
### Flow: Import
str_import_flow = 'M'
### Reporters list:
list_un_reporters = dict_codelist['reporterCode'].index.to_list()
### Partners list:
list_un_partners = dict_codelist['partnerCode'].index.to_list()
### Goods classification codes:
list_un_goods_ag2 = dict_codelist['clCode'][str_goods_type].index.to_list()
### Services classification codes:
list_un_services_ag2 = dict_codelist['clCode'][str_services_type].index.to_list()
#### United classification codes:
list_un_commodities_ag2 = list_un_goods_ag2 + list_un_services_ag2
### Years to collect data:
list_periods = list(map(str, range(date_start.year, date_end.year + 1)))
### Request tuning:
int_pause_short_sec = 10
int_pause_long_sec = 10
int_goods_period_portion = 5

In [11]:
### GOODS ONLY DATA LOADING ENGINE

gc.collect()
### Checking of file status & loading last observation saved:
if (os.path.exists(str_path_unc_res_all_annual)):
    ser_last_row = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res, start = -1)
    str_last_comm_id = ser_last_row.index[0][5]
    date_last_year = ser_last_row.index[0][0]
    bool_break_flag = True
    print('Last saved observation options:', date_last_year.year, '/', str_last_comm_id)
else:
    date_last_year = pd.to_datetime('1900-01-01')
    print(date_last_year.year)
    bool_break_flag = False
    
### Looping over period portions:
for iter_portion in range(-(-len(list_periods) // int_goods_period_portion)):
    gc.collect()
    ### Selecting periods:
    list_iter_periods = list_periods[iter_portion *  int_goods_period_portion : (iter_portion + 1) *  int_goods_period_portion]
    if (int(list_iter_periods[-1]) < date_last_year.year):
        continue
    else:         
        ### Commodities data loading:
        for iter_comm_id in list_un_goods_ag2:
            ### Starting point searching:
            if bool_break_flag:
                if (list_un_goods_ag2.index(str_last_comm_id) < list_un_goods_ag2.index(iter_comm_id)):
                    continue
                elif (list_un_goods_ag2.index(str_last_comm_id) == list_un_goods_ag2.index(iter_comm_id)):
                    bool_break_flag = False
                    continue
            ### Loading procedure:
            else:
                ### Container initialization:
                list_un_collection = []                
                ### Export Data Requests:                
                print(iter_comm_id, '/', list_iter_periods, '/ Export: Loading through the API')
                ### Export of Goods:
                df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_export_flow, list_un_reporters, list_un_partners, 
                                                       [iter_comm_id], list_iter_periods, str_primary_key)
                ### Adding dataset to container:
                if (df_iter_dataset is not None):
                    list_un_collection.append(df_iter_dataset)
                    time.sleep(int_pause_short_sec)
                else:
                    time.sleep(int_pause_long_sec) 
                ### Import Data Requests:
                print(iter_comm_id, '/', list_iter_periods, '/ Import: Loading through the API')
                ### Import of Goods:
                df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_import_flow, list_un_reporters, list_un_partners, 
                                                       [iter_comm_id], list_iter_periods, str_primary_key)
                ### Adding dataset to container:
                if (df_iter_dataset is not None):
                    list_un_collection.append(df_iter_dataset)
                    time.sleep(int_pause_short_sec)
                else:
                    time.sleep(int_pause_long_sec)             
                ### Downloaded Data concatenation and indexation:
                if (len(list_un_collection) > 0):
                    df_full_dataset = pd.concat(list_un_collection, axis = 0, sort = False, ignore_index = True)
                    ser_full_dataset = df_full_dataset.set_index(['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID']).squeeze().sort_index()
                    ### Dataset saving (need to be replaced with SQL Request):
                    ser_full_dataset.to_hdf(path_or_buf = str_path_unc_res_all_annual, key = str_key_unc_res, mode = 'a', format = 'table', complevel = 9, 
                                            append = True)
                    print(iter_comm_id, '/', list_iter_periods, ': Flows saved to database')
                else:
                    print(iter_comm_id, '/', list_iter_periods, ': Both flow\'s datasets are empty')
#            break
#    break

Last saved observation options: 2022 / 99


In [ ]:
### TEMP

df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB10']['fileuri']).json()['results'])
### Filtering needed level of services groups aggregation:
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
#display(df_eb_serv_ag2)
df_eb_serv_ag2.merge(df_eb_serv, left_on = 'parent', right_on = 'id', suffixes = ['_child', '_parent']).drop('parent_child', axis = 1)\
#              .to_excel('Data_Files/Test_Files/EBOPS_2010.xlsx')
#df_eb_serv[df_eb_serv['parent'] == '200']

In [8]:
### TEMP

### Service EBOPS Codes:
df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB10']['fileuri']).json()['results'])
### Filtering needed level of services groups aggregation:
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
df_eb_serv[df_eb_serv['id'] == '6.4']

,id,text,parent
120,6.4,6.4 Pension and standardized guarantee services,6


In [12]:
### TEMP

df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB']['fileuri']).json()['results'])
### Filtering needed level of services groups aggregation:
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
df_eb_serv_ag2

,id,text,parent
3,206,1.1 Sea transport,205
7,210,1.2 Air transport,205
11,214,1.3 Other transport,205
15,218,1.4 Other transport of which: Space transport,205
16,219,1.5 Other transport of which: Rail transport,205
20,223,1.6 Other transport of which: Road transport,205
24,227,1.7 Other transport of which: Inland waterway ...,205
28,231,1.8 Other transport of which: Pipeline transpo...,205
29,232,1.9 Other supporting an auxiliary transport se...,205
31,237,2.1 Business travel,236


In [16]:
### TEMP

int_period_portion = 5
str_services_class = 'EB02'

In [ ]:
### UN COMTRADE: GOODS DATA REQUEST PARAMETERS

### Primary key to authorize:
#str_primary_key = 'e690550ab9414234a6b705220596677a'
#str_primary_key = 'd79c218e2e9d464fade1810fd14347d8'
str_primary_key = '3f0b8d53d71b401e840d58c90a283176'
### Type: Goods
str_goods_type = 'C'
### Type: Services
str_services_type = 'S'
### Annual frequency:
str_freq = 'A'
### Goods classification:
str_goods_class = 'HS'
### Services classification classification:
str_services_class = 'EB'
### Flow: Export
str_export_flow = 'X'
### Flow: Import
str_import_flow = 'M'
### Reporters list:
list_un_reporters = dict_codelist['reporterCode'].index.to_list()
### Partners list:
list_un_partners = dict_codelist['partnerCode'].index.to_list()
### Goods classification codes:
list_un_goods_ag2 = dict_codelist['clCode'][str_goods_type].index.to_list()
### Services classification codes:
list_un_services_ag2 = dict_codelist['clCode'][str_services_type].index.to_list()
#### United classification codes:
list_un_commodities_ag2 = list_un_goods_ag2 + list_un_services_ag2
### Years to collect data:
list_periods = list(map(str, range(date_start.year, date_end.year + 1)))
### Request tuning:
int_pause_short_sec = 10
int_pause_long_sec = 10
int_goods_period_portion = 5

In [17]:
### GOODS ONLY DATA LOADING ENGINE

gc.collect()
### Checking of file status & loading last observation saved:
if (os.path.exists(str_path_unc_res_all_annual)):
    ser_last_row = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res, start = -1)
    str_last_comm_id = ser_last_row.index[0][5]
    date_last_year = ser_last_row.index[0][0]
    bool_break_flag = True
    print('Last saved observation options:', date_last_year.year, '/', str_last_comm_id)
else:
    date_last_year = pd.to_datetime('1900-01-01')
    print(date_last_year.year)
    bool_break_flag = False
    
### Looping over period portions:
for iter_portion in range(-(-len(list_periods) // int_goods_period_portion)):
    gc.collect()
    ### Selecting periods:
    list_iter_periods = list_periods[iter_portion *  int_goods_period_portion : (iter_portion + 1) *  int_goods_period_portion]
    if (int(list_iter_periods[-1]) < date_last_year.year):
        continue
    else:         
        ### Commodities data loading:
        for iter_comm_id in list_un_goods_ag2:
            ### Starting point searching:
            if bool_break_flag:
                if (list_un_goods_ag2.index(str_last_comm_id) < list_un_goods_ag2.index(iter_comm_id)):
                    continue
                elif (list_un_goods_ag2.index(str_last_comm_id) == list_un_goods_ag2.index(iter_comm_id)):
                    bool_break_flag = False
                    continue
            ### Loading procedure:
            else:
                ### Container initialization:
                list_un_collection = []                
                ### Export Data Requests:                
                print(iter_comm_id, '/', list_iter_periods, '/ Export: Loading through the API')
                ### Export of Goods:
                df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_export_flow, list_un_reporters, list_un_partners, 
                                                       [iter_comm_id], list_iter_periods, str_primary_key)
                ### Adding dataset to container:
                if (df_iter_dataset is not None):
                    list_un_collection.append(df_iter_dataset)
                    time.sleep(int_pause_short_sec)
                else:
                    time.sleep(int_pause_long_sec) 
                ### Import Data Requests:
                print(iter_comm_id, '/', list_iter_periods, '/ Import: Loading through the API')
                ### Import of Goods:
                df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_import_flow, list_un_reporters, list_un_partners, 
                                                       [iter_comm_id], list_iter_periods, str_primary_key)
                ### Adding dataset to container:
                if (df_iter_dataset is not None):
                    list_un_collection.append(df_iter_dataset)
                    time.sleep(int_pause_short_sec)
                else:
                    time.sleep(int_pause_long_sec)             
                ### Downloaded Data concatenation and indexation:
                if (len(list_un_collection) > 0):
                    df_full_dataset = pd.concat(list_un_collection, axis = 0, sort = False, ignore_index = True)
                    ser_full_dataset = df_full_dataset.set_index(['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID']).squeeze().sort_index()
                    ### Dataset saving (need to be replaced with SQL Request):
                    ser_full_dataset.to_hdf(path_or_buf = str_path_unc_res_all_annual, key = str_key_unc_res, mode = 'a', format = 'table', complevel = 9, 
                                            append = True)
                    print(iter_comm_id, '/', list_iter_periods, ': Flows saved to database')
                else:
                    print(iter_comm_id, '/', list_iter_periods, ': Both flow\'s datasets are empty')
#            break
#    break

Last saved observation options: 2022 / 99


In [64]:
### TEMP

dict_test = {}
dict_test['2010_EB'] = df_iter_dataset
#dict_test['2010_EB10'] = df_iter_dataset
#dict_test['2010_EB2'] = df_iter_dataset
#print(len(dict_test['2010_EB10']))
#print(len(dict_test['2010_EB']))
#print(len(set(dict_test['2010_EB10'].set_index(['Reporter', 'Partner', 'Flow'])['Value'].index) - \
#set(dict_test['2010_EB'].set_index(['Reporter', 'Partner', 'Flow'])['Value'].index)))
#print(len(set(dict_test['2010_EB'].set_index(['Reporter', 'Partner', 'Flow'])['Value'].index) - \
#set(dict_test['2010_EB10'].set_index(['Reporter', 'Partner', 'Flow'])['Value'].index)))

In [62]:
### TEMP

display(dict_test['2010_EB10']['Reporter'].unique())
display(dict_test['2010_EB']['Reporter'].unique())

[AL, AT, BE, HR, DK, ..., ME, NL, PL, RS, SE]
Length: 14
Categories (14, object): [AL < AT < BE < DK ... NL < PL < RS < SE]

[BY, RU, UA, US]
Categories (4, object): [BY < RU < UA < US]

In [62]:
### TEMP

#ser_unc_export = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res,
#                                     where = "(Flow = 'Export') & (Type == 'Services')").droplevel('Flow')

ser_unc_export['2015-12-31']

Reporter  Partner  Type      Commodity_ID
BT        IN       Services  210             17318
                             237              5702
                             257              1763
                             292               354
Name: Value, dtype: int32

In [18]:
### DATA CONCATENATION AND SAVING

#### Downloaded Data concatenation and indexation:
#df_full_dataset = pd.concat(list_un_collection, axis = 0, sort = False, ignore_index = True)
#del list_un_collection
#ser_full_dataset = df_full_dataset.set_index(['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID']).squeeze().sort_index()
#### Dataset saving (need to be replaced with SQL Request):
#ser_full_dataset.to_hdf(path_or_buf = str_path_unc_res_all_annual, key = str_key_unc_res, mode = 'w', format = 'table', complevel = 9)

In [ ]:
### EXPORT AND REVERTED IMPORT CONCATENATION

gc.collect()
### File deleting (need to be replaced with SQL Request):
if (os.path.exists(str_path_unc_res_flows)):
    os.remove(str_path_unc_res_flows)
### Results container:
list_export_aug = []
### Countries portion length:
int_portion = 5
list_unc_countries = sorted(dict_codelist['partnerCode'].unique())
### Looping over countries portions:
for iter_num in range(len(list_unc_countries) // int_portion + 1):
    gc.collect()    
    ### Portion of countries selecting:
    list_iter_countries = list(list_unc_countries)[int_portion * iter_num : int_portion * (iter_num + 1)]
    if (len(list_iter_countries) > 0):
        print(list_iter_countries)
        ### Export data loading:
        ser_unc_export = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res,
                                     where = "(Flow = 'Export') & (Reporter in list_iter_countries) & (Partner != 'World')").droplevel('Flow')
        print('Export dataset loaded')
        ### Import data loading:
        ser_unc_import = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res, 
                                     where = "(Flow = 'Import') & (Partner in list_iter_countries)").droplevel('Flow')
        print('Import dataset loaded')    
        ### Import data reverting:
        ser_unc_import.index.set_names('Partner_Inv', level = 1, inplace = True)
        ser_unc_import.index.set_names('Reporter', level = 2, inplace = True)
        ser_unc_import.index.set_names('Partner', level = 1, inplace = True)
        ser_unc_import = ser_unc_import.swaplevel('Reporter', 'Partner').sort_index()
        print('Import dataset reverted')
        ### Datasets concatenation:
        df_export_aug = pd.concat([ser_unc_export, ser_unc_import], axis = 1, names = 'Source Flow', keys = ['Export', 'Import']).astype('float32')
        del ser_unc_export
        del ser_unc_import    
        gc.collect()    
        print('Export and reverted Import dataset concatenated')
        ### Aggregateddataset saving (need to be replaced with SQL Request):
        df_export_aug.to_hdf(str_path_unc_res_flows, key = str_key_unc_res, mode = 'a', format = 'table', complevel = 9, append = True)                
        print('Aggregated dataset added to database')

In [ ]:
### CIF COEFFICIENTS CALCULATION & IMPLEMENTATION

gc.collect()
### Files deleting (need to be replaced with SQL Request):
if (os.path.exists(str_path_export_bilateral)):
    os.remove(str_path_export_bilateral)
if (os.path.exists(str_path_import_bilateral)):
    os.remove(str_path_import_bilateral)
### Getting full list of commodities:
str_date = '2020-12-31'
list_commodity_id = sorted(pd.read_hdf(str_path_unc_res_flows, key = str_key_unc_res, where = "Date in str_date").index.get_level_values('Commodity_ID').unique())
### Bounds to filter bilateral Import to Export ratio before median calculation:
flo_lower_bound = 1.0
flo_upper_bound = 2.0
### Bilateral median calculation procedure:
def get_obs_median(df_comm):
    ### Export to Import ratio:
    ser_obs_coeff = df_comm['Import'] / df_comm['Export']
    ### Ratio filtering:
    ser_obs_coeff = ser_obs_coeff.loc[(ser_obs_coeff >= flo_lower_bound) & (ser_obs_coeff <= flo_upper_bound)]
    ### Filtered timeseries median as a result:
    return ser_obs_coeff.median()
### Calulation CIF coefficient for each commodity:
for iter_commodity in list_commodity_id:
    gc.collect()
    ### Commodity flows loading (need to be replaced with SQL Request):
    df_iter_flows = pd.read_hdf(str_path_unc_res_flows, key = str_key_unc_res, where = "Commodity_ID = iter_commodity")
    ### Bilateral Commodity CIF Median calculation:
    ser_cif_median = df_iter_flows.droplevel('Commodity_ID').groupby(['Reporter', 'Partner']).apply(get_obs_median)
    ### General Commodity Median calculation:
    flo_median = ser_cif_median.median()
    print(iter_commodity, ':', flo_median)
    ### Filling missed bilateral values with general commodity median:
    if not (np.isnan(flo_median)):
        ser_cif_median.fillna(flo_median, inplace = True)        
    ser_cif_median.name = 'CIF_Coefficient'              
    ### Adding CIF coefficients to dataset:
    df_export_cif = df_iter_flows.merge(ser_cif_median, left_index = True, right_index = True)
    df_export_cif = df_export_cif.reorder_levels(['Date', 'Reporter', 'Partner', 'Type', 'Commodity_ID'])
    ### Import correction:
    df_export_cif['Import_Corrected'] = df_export_cif['Import'] / df_export_cif['CIF_Coefficient']
    ### Export correction:
    df_export_cif['Export_Corrected'] = df_export_cif['Export'] * df_export_cif['CIF_Coefficient']
    ### Combining Export & Import data:
    ser_export_cif = df_export_cif['Export'].combine_first(df_export_cif['Import_Corrected']).astype('float32')
    ser_import_cif = df_export_cif['Import'].combine_first(df_export_cif['Export_Corrected']).astype('float32')
    gc.collect()
    ### Import data reverting back to original order:
    ser_import_cif = ser_import_cif.reorder_levels(['Date', 'Partner', 'Reporter', 'Type', 'Commodity_ID']).sort_index()                               
    ser_import_cif.index.names = ['Date', 'Reporter', 'Partner', 'Type', 'Commodity_ID']
    gc.collect()
    ### Augmented flows saving (need to be replaced with SQL Request):
    ser_export_cif.squeeze().to_hdf(str_path_export_bilateral, key = str_key_unc_export, mode = 'a', format = 'table', complevel = 9, append = True, 
                                    min_itemsize = {'Type': 8, 'Commodity_ID': 3})
    ser_import_cif.squeeze().to_hdf(str_path_import_bilateral, key = str_key_unc_import, mode = 'a', format = 'table', complevel = 9, append = True, 
                                    min_itemsize = {'Type': 8, 'Commodity_ID': 3}) 